In [9]:
import os
import scipy.signal as signal
import numpy as np
import matplotlib.pyplot as plt
#from  scipy.signal import ShortTimeFFT
#import scipy.signal.windows as wins
import pickle


cwd = os.getcwd()
base_folder="Dataset_848"
subfolders = [f.path for f in os.scandir(base_folder) if f.is_dir() ]
filenames=[]

for folder in subfolders:
    files = [f.path for f in os.scandir(folder) if f.is_file()]
    filenames.extend(files)

In [7]:
!pip install scipy

  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl (46.2 MB)


$f_0$=5.8GHz
BW=400MHz
$\tau_s$=0.001s

In [11]:
# Only save subjects id
subjects = []
for filename in filenames:
    name_of_file = filename.split('\\')[-1][:-4]
    subject_id = int(name_of_file.split('P')[1].split('A')[0])
    subjects.append(subject_id)
subjects = np.array(subjects)
subjects.shape
np.save('subjects_array.npy', subjects)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]


In [5]:
#parameters
t_sweep=0.001
n_rangebins=128
bandwidth=4e8

winlength=128
overlap=0.5
nfft=2048
win=wins.hann(winlength)
hop=int(winlength*(1-overlap))

spectrograms=[]
labels=[]

b, a = signal.butter(4,0.0075, 'high') # filter coefficients for MTI filter

for filename in filenames: #iterate over the whole dataset
    #get the label based on
    name_of_file = filename.split('\\')[-1][:-4]
    activity = int(name_of_file.split('P')[0]) 
   
    labels.append(activity)

    with open(filename, 'r') as file:
        data = np.loadtxt(file, delimiter=' ', skiprows=4, dtype=np.complex_, converters=lambda s: complex(s.decode().replace('i', 'j')))
        data_numsamples=len(data)
        data_time=data_numsamples*t_sweep/n_rangebins
        fs=int(n_rangebins/t_sweep)
        n_sweeps=data_time/t_sweep

        data = data.reshape(n_rangebins, int(n_sweeps), order='F') #reshape data from vector to matrix of size rangebins x sweeps

        data_fft = np.fft.fftshift(np.fft.fft(data, axis=0), axes=0) #perform fft on each sweep to convert beat frequency to range returns
        data_fft = data_fft[int(n_rangebins/2):,:]# keep only the bottom half since fft is symmetric
        range_profile = signal.filtfilt(b, a, data_fft, axis=1)# MTI filter
        prof=np.sum(range_profile,axis=0) #sum all range returns to get the profile

        stft=ShortTimeFFT(win, hop, mfft=nfft, fs=fs, scale_to='magnitude', fft_mode='centered')

        Sxx = stft.spectrogram(prof,axis=0) #compute the spectrogram
        nul,nul,f_lo,f_hi=stft.extent(int(fs*n_sweeps))

        spectrograms.append(Sxx)

    '''# range and time axes
    freq_axis=np.linspace(0,n_rangebins,n_rangebins)*fs/(2*n_rangebins)
    range_axis=freq_axis*3e8*t_sweep/(2*bandwidth)
    time_axis=np.arange(0, data_time, t_sweep)'''  
    
    # check progress every 50 samples 
    if len(labels)%50==0:
        print(len(labels))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750


NameError: name 'pickle' is not defined

In [2]:
#save the data
with open('spectrograms.pkl', 'wb') as f:
    pickle.dump(spectrograms, f)
with open('labels.pkl', 'wb') as f:
    pickle.dump(labels, f)

NameError: name 'spectrograms' is not defined

In [10]:
spectrograms=pickle.load(open('spectrograms.pkl', 'rb'))
labels=pickle.load(open('labels.pkl', 'rb'))

#check length of spectrograms and labels
print(len(spectrograms))
print(len(labels))

#join the spectrograms and labels
dataset = list(zip(spectrograms, labels))

EOFError: Ran out of input

In [11]:
# split the dataset according to the second dimension of the first element of the tuple
longer=[]
long=[]
short=[]

for i in range(len(dataset)):
    if dataset[i][0].shape[1]==158:
        long.append(dataset[i])
    elif dataset[i][0].shape[1]==80:
        short.append(dataset[i])
    elif dataset[i][0].shape[1]==314:
        longer.append(dataset[i])
    else:
        print('sample number:', i, 'shape:', dataset[i][0].shape)
        raise ValueError('Unexpected number of time samples')

print(len(longer))
print(len(long))
print(len(short))

## save the lists to files for later use
with open('longer.pkl', 'wb') as f:
    pickle.dump(longer, f)

with open('long.pkl', 'wb') as f:
    pickle.dump(long, f)

with open('short.pkl', 'wb') as f:
    pickle.dump(short, f)

8
305
1441


## decimating the longer walking samples

In [17]:
from scipy.signal import resample

dataset_resampled=[]
for i in range(len(dataset)):
    if dataset[i][0].shape[1]!=80:
        dataset_resampled.append((resample(dataset[i][0], 80, axis=1), dataset[i][1]))
    else:
        dataset_resampled.append(dataset[i])


In [23]:
spec_resampled, labels = zip(*dataset_resampled)

spec_resampled_array=np.array(spec_resampled)
labels_array=np.array(labels)

print(spec_resampled_array.shape)
print(labels_array.shape)

#save the resampled data
np.save('spec_resampled_array.npy', spec_resampled_array)
np.save('labels_array.npy', labels_array)


(1754, 2048, 80)
(1754,)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1754, 2) + inhomogeneous part.

## padding

In [9]:
dataset_padded=[]
for i in range(len(dataset)):
    dataset_padded.append((np.pad(dataset[i][0], ((0,0),(0,314-dataset[i][0].shape[1]))), dataset[i][1]))

In [12]:
padded_array,labels_array=zip(*dataset_padded)
padded_array=np.array(padded_array)
labels_array=np.array(labels_array)

In [13]:
print(padded_array.shape)
print(labels_array.shape)

print(np.unique(labels_array))

(1754, 2048, 314)
(1754,)
[1 2 3 4 5 6]


### test stuff to look for longer samples

In [21]:
files_feb= [f.path for f in os.scandir('Dataset_848\\6 February 2019 NG Homes Dataset\\') if f.is_file()]
print(files_feb)

ranges_feb=[]

for filename in files_feb[:20]:
    with open(filename, 'r') as file:
        data = np.loadtxt(file, delimiter=' ', skiprows=4, dtype=np.complex_, converters=lambda s: complex(s.decode().replace('i', 'j')))
        data_numsamples=len(data)
        data_time=data_numsamples*t_sweep/n_rangebins
        fs=int(n_rangebins/t_sweep)
        n_sweeps=data_time/t_sweep

        data = data.reshape(n_rangebins, int(n_sweeps), order='F')

        data_fft = np.fft.fftshift(np.fft.fft(data, axis=0), axes=0)
        data_fft = data_fft[int(n_rangebins/2):,:]
        range_profile = signal.filtfilt(b, a, data_fft, axis=1)

        prof=np.sum(range_profile,axis=0)

        ranges_feb.append(range_profile)

for i in range(len(ranges_feb)):
    plt.figure()
    plt.imshow(20*np.log(np.abs(ranges_feb[i])/np.max(np.abs(ranges_feb[i]))), aspect='auto', cmap='jet')
    plt.colorbar()
    plt.title('Range profile of the '+str(i)+'th sweep of the 6th of February dataset')
    plt.xlabel('Sweeps')
    plt.ylabel('Range bins')
    plt.show()


['Dataset_848\\6 February 2019 NG Homes Dataset\\1P08A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P08A01R2.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P08A01R3.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P18A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P18A01R2.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P18A01R3.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P19A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P19A01R2.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P19A01R3.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P20A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P20A01R2.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P20A01R3.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P21A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P21A01R2.dat', 'Dataset_848\\6 February 2019 NG Homes Dataset\\1P22A01R1.dat', 'Dataset_848\\6 February 2019 NG Homes 